##### Assignment 2: Designing AI Solutions: Rapid Prototyping with and without Generative AI Tools

##### Importing the required libraries 

##### These are the required Python libraries for data analysis, visualization, and model building.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, r2_score


##### Loding the Dataset

##### Load the Operating Room Utilization dataset and inspect its structure and columns.

In [2]:
file_path = "data/2022_Q1_OR_Utilization.csv"  # Adjust path if needed
df = pd.read_csv(file_path)

print("Dataset Loaded Successfully")
print(df.head())
print("\nColumns:", df.columns.tolist())


Dataset Loaded Successfully
   index  Encounter ID      Date  OR Suite      Service  CPT Code  \
0      0         10001  01/03/22         1     Podiatry     28110   
1      1         10002  01/03/22         1     Podiatry     28055   
2      2         10003  01/03/22         1     Podiatry     28297   
3      3         10004  01/03/22         1     Podiatry     28296   
4      4         10005  01/03/22         2  Orthopedics     27445   

                             CPT Description  Booked Time (min)  \
0   Partial ostectomy, fifth metatarsal head                 90   
1  Neurectomy, intrinsic musculature of foot                 60   
2                       Lapidus bunionectomy                150   
3         Bunionectomy with distal osteotomy                120   
4        Arthroplasty, knee, hinge prothesis                120   

         OR Schedule          Wheels In         Start Time           End Time  \
0  01/03/22 07:00 AM  01/03/22 07:05 AM  01/03/22 07:32 AM  01/03/22 09:0

##### Data Preprocessing

###### Clean and prepare the dataset:

###### Convert time columns to datetime

###### Create duration-based features

###### Extract useful time info (like start hour, day of week)

###### Remove missing and invalid records

In [3]:
time_cols = ['Start Time', 'End Time', 'Wheels In', 'Wheels Out']
for col in time_cols:
    df[col] = pd.to_datetime(df[col], errors='coerce')

df['Actual OR Duration (min)'] = (df['End Time'] - df['Start Time']).dt.total_seconds() / 60
df['Setup Duration (min)'] = (df['Start Time'] - df['Wheels In']).dt.total_seconds() / 60
df['Room Occupancy (min)'] = (df['Wheels Out'] - df['Wheels In']).dt.total_seconds() / 60
df['Start Hour'] = df['Start Time'].dt.hour
df['Day of Week'] = df['Start Time'].dt.day_name()

df = df.dropna(subset=['Actual OR Duration (min)', 'Booked Time (min)', 'Service', 'CPT Description', 'OR Suite'])
df = df[df['Actual OR Duration (min)'] > 0]

print("\n Data Preprocessed Successfully")
print(df[['Service', 'Booked Time (min)', 'Actual OR Duration (min)']].head())


C:\Users\User\AppData\Local\Temp\ipykernel_23412\3094064281.py:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df[col] = pd.to_datetime(df[col], errors='coerce')
C:\Users\User\AppData\Local\Temp\ipykernel_23412\3094064281.py:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df[col] = pd.to_datetime(df[col], errors='coerce')
C:\Users\User\AppData\Local\Temp\ipykernel_23412\3094064281.py:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df[col] = pd.to_datetime(df[col], errors='coerce')
C:\Users\User\AppData\Local\Temp\ipykernel_23412\3094064281.py:3: UserWarning: Could not in


 Data Preprocessed Successfully
       Service  Booked Time (min)  Actual OR Duration (min)
0     Podiatry                 90                      93.0
1     Podiatry                 60                      48.0
2     Podiatry                150                      22.0
3     Podiatry                120                      57.0
4  Orthopedics                120                     108.0


##### Feature Selection

##### Select input features (X) and the target variable (y) for model training.

In [4]:
features = ['Service', 'CPT Description', 'Booked Time (min)', 'OR Suite', 'Start Hour', 'Day of Week']
target = 'Actual OR Duration (min)'

X = df[features]
y = df[target]
